#Выгрузка вакансий по запросу с сайта hh.ru

Подготовим папки для загрузки вакансий

In [ ]:
!rm -rf /content/pars_vacancies
!mkdir /content/pars_vacancies
!rm -rf /content/vacancies
!mkdir /content/vacancies

Скачаем с сайта hh.ru вакансии. Ниже приведен код, для скачивания вакансий по запросу "Программист"

In [ ]:
# Библиотека работы с HTTP-запросами
import requests 

# Пакет для работы с JSDN-файлами
import json

# Модуль для работы со значением времени
import time

# Модуль для работы с операционной системой
import os

def getPages(page = 0):
    '''Функция запроса страницы.
       page - Индекс страницы
       (по умолчанию начинается с первой страницы)'''

       # Справочник для переменной GET-запроса
       # Значения для указания значений в справочнике 
       # указаны в документации к API hh.ru по ссылке: 
       # https://github.com/hhru/api
    params = {
                'text': 'Аналитик данных',
                'area': 1,
                'page': page,
                'per_page': 100}

    req = requests.get('https://api.hh.ru/vacancies', params)
    data = req.content.decode()
    req.close()
    return data

        

# Считаем первые Х вакансий
for page in range(0,20):
    jsObj = json.loads(getPages(page))

    # Сохраняем файлы в папку {путь до текущего документа со скриптом}\установить\ваш\путь
    # Определяем количество файлов в папке для сохранения документа с ответом запроса
    # Полученное значение используем для формирования имени документа
    nextFileName = '/content/pars_vacancies/{}.json'.format(len(os.listdir('/content/pars_vacancies')))

    # Создаем новый документ, записываем в него ответ запроса, после закрываем
    f = open(nextFileName, mode='w', encoding='utf8')
    f.write(json.dumps(jsObj, ensure_ascii=False, indent=3))

    # Проверка на последнюю страницу, если вакансий меньше 2000
    if (jsObj['pages'] - page) <= 1:
        break

    time.sleep(0.25)

print('Страницы поиска собраны!')

На прошлом шаге были получены документы с большим количеством вакансий. Однако они являются неполными и большая часть их описания отсутствует. Обратимся к каждой вакансии по ссылке, соберем недостающую информацию и сохраним каждую вакансию в отдельный файл.

In [ ]:
import json
import os
import requests
import time

# Получаем перечень ранее созданных файлов со списком вакансий и проходимся по нему в цикле
for fl in os.listdir('/content/pars_vacancies'):
	f = open('/content/pars_vacancies/{}'.format(fl), encoding='utf8')
	jsonText = f.read()
	f.close()

	jsonObj = json.loads(jsonText)

	for vacancies in jsonObj['items']:
		req = requests.get(vacancies['url'])
		data = req.content.decode()
		req.close()

		fileName = '/content/vacancies/{}.json'.format(vacancies['id'])
		file = open(fileName, mode='w', encoding='utf8')
		file.write(data)
		file.close()

print('Вакансии собраны')

Сформируем один csv файл для всех вакансий, в котором будет отражен id вакансии, название вакансии, требования к кандидату и описание вакансии 

In [ ]:
import pandas as pd
import json
import os

# Создание списков для столбцов таблицы vacancies
IDs = [] # Список идентификаторов вакансий
names = [] # Список наименований вакансий

# Создание списков для столбцов таблицы skills
skills_vac = [] # Список идентификаторов вакансий
skills_name = [] # Список идентификаторов вакансий
descr = [] # Список названий навыков

# Список для описания вакансии



cnt_docs = len(os.listdir('/content/vacancies'))
i = 0
# Проходимся по всем файлам в папке vacancies
for fl in os.listdir('/content/vacancies'):
    
    # Открываем, читаем и закрываем файл
    f = open('/content/vacancies/{}'.format(fl), encoding='utf8')
    jsonText = f.read()
    f.close()
 
    # Текст файла переводим в справочник
    jsonObj = json.loads(jsonText)
 
    # Заполняем списки для таблиц
    IDs.append(jsonObj['id'])
    names.append(jsonObj['name'])

    #Описание вакансии


    # Т.к. навыки хранятся в виде массива, то проходимся по нему циклом
    skills_name = []
    for skl in jsonObj['key_skills']:
        skills_vac.append(jsonObj['id'])
        skills_name.append(skl['name'])
    skills_vac.append(skills_name)
    descr.append(jsonObj['description'])

# Создание датафрейма, который содержит в себе информацию о названии вакансии, требованиях к кандидату и описании самой вакансии
df = pd.DataFrame({'id': IDs, 'name': names, 'skills': skills_vac, 'description': descr})
df.to_csv('anal_data.csv')
#df.to_excel('programmer.xlsx')
print('OK') 